<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/R_Image_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data
# #Training
# # #Labels
# # #Images
# #Validation
# # #Labels
# # #Images

Install Packages

In [1]:
install.packages(c('raster','sf','fasterize'))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘e1071’, ‘sp’, ‘Rcpp’, ‘terra’, ‘classInt’, ‘s2’, ‘units’, ‘wk’, ‘RcppArmadillo’




In [2]:
install.packages("googledrive")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
install.packages("devtools")

devtools:::install_github("gearslaboratory/gdalUtils")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘promises’, ‘shiny’, ‘downlit’, ‘htmlwidgets’, ‘miniUI’, ‘pkgdown’, ‘profvis’, ‘urlchecker’





R.methodsS3 (NA -> 1.8.2 ) [CRAN]
R.oo        (NA -> 1.26.0) [CRAN]
iterators   (NA -> 1.0.14) [CRAN]
R.utils     (NA -> 2.12.3) [CRAN]
foreach     (NA -> 1.5.2 ) [CRAN]


Skipping 1 packages not available: rgdal

Installing 6 packages: R.methodsS3, R.oo, iterators, rgdal, R.utils, foreach

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘rgdal’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpfLXr9E/remotes27778cb5ddd/gearslaboratory-gdalUtils-8872aaf/DESCRIPTION’ ... OK
* preparing ‘gdalUtils’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘gdalUtils_2.0.3.2.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in i.p(...):
“installation of package ‘/tmp/RtmpfLXr9E/file27732a12eb7/gdalUtils_2.0.3.2.tar.gz’ had non-zero exit status”


Load Libraries

In [4]:
library(raster)
library(sf)
library(fasterize)
library(magrittr)
library(googledrive)
library(parallel)
library(data.table)
library(stringr)
# library(gdalUtils)

Loading required package: sp

Linking to GEOS 3.11.1, GDAL 3.6.4, PROJ 9.1.1; sf_use_s2() is TRUE


Attaching package: ‘fasterize’


The following object is masked from ‘package:graphics’:

    plot


The following object is masked from ‘package:base’:

    plot



Attaching package: ‘magrittr’


The following object is masked from ‘package:raster’:

    extract



Attaching package: ‘data.table’


The following object is masked from ‘package:raster’:

    shift




Connect to Google Drive

In [5]:
drive_auth(use_oob = TRUE, cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=b576452c70a320ab320811abee1b8dd7&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQWZKb2hYbDdrRW1YX1lobkhwa0xsLWlDbkhrMjUzWHZkUzF5bkdQXzFPRXVydk1wS0h6STJvcDRQaFRmVXRYVG1fQmtCUSIsInN0YXRlIjoiYjU3NjQ1MmM3MGEzMjBhYjMyMDgxMWFiZWUxYjhkZDcifQ==


In [6]:
dir.create('input_data')

In [7]:
# https://drive.google.com/file/d/1thj0vGXN0xgXxI_XDY-hagp-gWHZCdaS/view?usp=drive_link
drive_download(file = as_id('1thj0vGXN0xgXxI_XDY-hagp-gWHZCdaS'), path =  '/content/input_data/naip_index.zip', overwrite = FALSE)
unzip(zipfile ='/content/input_data/naip_index.zip', exdir = "/content/input_data/naip_index/")
naip_index_gpkg = st_read('/content/input_data/naip_index/tileindex_HI_NAIP_2021.shp')

File downloaded:

• tileindex_HI_NAIP_2021.zip <id: 1thj0vGXN0xgXxI_XDY-hagp-gWHZCdaS>

Saved locally as:

• /content/input_data/naip_index.zip



Reading layer `tileindex_HI_NAIP_2021' from data source 
  `/content/input_data/naip_index/tileindex_HI_NAIP_2021.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 611 features and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -160.3144 ymin: 18.8732 xmax: -154.7481 ymax: 22.25184
Geodetic CRS:  NAD83


In [8]:
# NWI wetlands for all islands
# https://drive.google.com/file/d/1ozE_Ldpio1LJaH2WS9r9hqRAIBDV8TDF/view?usp=drive_link


# NWI wetlands for the islands in UTM 4 (kauai, oahu, molokai, maui)
# https://drive.google.com/file/d/1uKY_cwL8P_bir8T4VY3f-gKs4ZQQyYk9/view?usp=drive_link
drive_download(file = as_id('1uKY_cwL8P_bir8T4VY3f-gKs4ZQQyYk9'), path =  '/content/input_data/NWI_wetlands.gpkg', overwrite = TRUE)
nwi_gpkg = st_read('/content/input_data/NWI_wetlands.gpkg')

File downloaded:

• nwi_wetlands_4.gpkg <id: 1uKY_cwL8P_bir8T4VY3f-gKs4ZQQyYk9>

Saved locally as:

• /content/input_data/NWI_wetlands.gpkg



Reading layer `nwi_wetlands' from data source 
  `/content/input_data/NWI_wetlands.gpkg' using driver `GPKG'
Simple feature collection with 10866 features and 6 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 412977.7 ymin: 2263072 xmax: 820600.7 ymax: 2464465
Projected CRS: WGS 84 / UTM zone 4N


In [9]:
# AOI wetlands for all islands
# https://drive.google.com/file/d/1cKHJ95tAkl0CSoNZkdO0w2wlaH6Wr_YZ/view?usp=drive_link
# drive_download(file = as_id('1cKHJ95tAkl0CSoNZkdO0w2wlaH6Wr_YZ'), path =  '/content/input_data/wetlands.gpgk', overwrite = TRUE)


# AOI wetlands for the islands in UTM 4 (kauai, oahu, molokai, maui)
# https://drive.google.com/file/d/1AsyHxmJ7MUJHwEqXUG61OXQ6bz9mgvDH/view?usp=drive_link
drive_download(file = as_id('1AsyHxmJ7MUJHwEqXUG61OXQ6bz9mgvDH'), path =  '/content/input_data/wetlands.gpgk', overwrite = TRUE)
wetlands_gpkg = st_read('/content/input_data/wetlands.gpgk')

File downloaded:

• aoi_wetland_4.gpkg <id: 1AsyHxmJ7MUJHwEqXUG61OXQ6bz9mgvDH>

Saved locally as:

• /content/input_data/wetlands.gpgk

Warning message in CPL_read_ogr(dsn, layer, query, as.character(options), quiet, :
“GDAL Message 1: File /content/input_data/wetlands.gpgk has GPKG application_id, but non conformant file extension”


Reading layer `aoi_wetland_4' from data source `/content/input_data/wetlands.gpgk' using driver `GPKG'
Simple feature collection with 16 features and 6 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 444046 ymin: 2299914 xmax: 766191.9 ymax: 2457336
Projected CRS: WGS 84 / UTM zone 4N


In [10]:
# NLCD for all islands
# https://drive.google.com/file/d/1foSde5txo6hPvWp4kcQF8We2WTDqDpYh/view?usp=drive_link
drive_download(file = as_id('1foSde5txo6hPvWp4kcQF8We2WTDqDpYh'), path =  '/content/input_data/nlcd_hi_2001', overwrite = TRUE)
nlcd_tif <- raster('/content/input_data/nlcd_hi_2001')

File downloaded:

• nlcd_hi_2001.tif <id: 1foSde5txo6hPvWp4kcQF8We2WTDqDpYh>

Saved locally as:

• /content/input_data/nlcd_hi_2001



In [11]:
download_index_gpkg <- naip_index_gpkg %>% st_intersection(wetlands_gpkg %>% st_transform(st_crs(naip_index_gpkg)))

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”


In [12]:
download_index_gpkg$url %>% unique %>% length

[1] 50

In [22]:
download_index_gpkg$path

[1] "E:/Data/AOI_Shapefiles/kauai/anahola.shp"       
 [2] "E:/Data/AOI_Shapefiles/kauai/anahola.shp"       
 [3] "E:/Data/AOI_Shapefiles/oahu/haleiwa.shp"        
 [4] "E:/Data/AOI_Shapefiles/oahu/haleiwa.shp"        
 [5] "E:/Data/AOI_Shapefiles/oahu/haleiwa.shp"        
 [6] "E:/Data/AOI_Shapefiles/oahu/haleiwa.shp"        
 [7] "E:/Data/AOI_Shapefiles/kauai/hanalei.shp"       
 [8] "E:/Data/AOI_Shapefiles/kauai/hanalei.shp"       
 [9] "E:/Data/AOI_Shapefiles/kauai/hanalei.shp"       
[10] "E:/Data/AOI_Shapefiles/kauai/hanalei.shp"       
[11] "E:/Data/AOI_Shapefiles/kauai/hanalei.shp"       
[12] "E:/Data/AOI_Shapefiles/kauai/huleia.shp"        
[13] "E:/Data/AOI_Shapefiles/kauai/huleia.shp"        
[14] "E:/Data/AOI_Shapefiles/kauai/huleia.shp"        
[15] "E:/Data/AOI_Shapefiles/kauai/huleia.shp"        
[16] "E:/Data/AOI_Shapefiles/kauai/huleia.shp"        
[17] "E:/Data/AOI_Shapefiles/oahu/kahuku.shp"         
[18] "E:/Data/AOI_Shapefiles/oahu/kahuku.shp"         
[19] "E:/Data/AOI_Shapefiles/oahu/kahuku.shp"         
[20] "E:/Data/AOI_Shapefiles/oahu/kahuku.shp"         
[21] "E:/Data/AOI_Shapefiles/molokai/kakahaia.shp"    
[22] "E:/Data/AOI_Shapefiles/molokai/kakahaia.shp"    
[23] "E:/Data/AOI_Shapefiles/molokai/kakahaia.shp"    
[24] "E:/Data/AOI_Shapefiles/molokai/kakahaia.shp"    
[25] "E:/Data/AOI_Shapefiles/molokai/kaluaapuhi.shp"  
[26] "E:/Data/AOI_Shapefiles/molokai/kaluaapuhi.shp"  
[27] "E:/Data/AOI_Shapefiles/molokai/kaluaapuhi.shp"  
[28] "E:/Data/AOI_Shapefiles/molokai/kamalo.shp"      
[29] "E:/Data/AOI_Shapefiles/molokai/kamalo.shp"      
[30] "E:/Data/AOI_Shapefiles/molokai/kamalo.shp"      
[31] "E:/Data/AOI_Shapefiles/molokai/kamalo.shp"      
[32] "E:/Data/AOI_Shapefiles/maui/kanaha.shp"         
[33] "E:/Data/AOI_Shapefiles/maui/kanaha.shp"         
[34] "E:/Data/AOI_Shapefiles/oahu/kawainui_marsh.shp" 
[35] "E:/Data/AOI_Shapefiles/oahu/kawainui_marsh.shp" 
[36] "E:/Data/AOI_Shapefiles/oahu/kawainui_marsh.shp" 
[37] "E:/Data/AOI_Shapefiles/oahu/kawainui_marsh.shp" 
[38] "E:/Data/AOI_Shapefiles/maui/kealia.shp"         
[39] "E:/Data/AOI_Shapefiles/maui/kealia.shp"         
[40] "E:/Data/AOI_Shapefiles/kauai/lumahai.shp"       
[41] "E:/Data/AOI_Shapefiles/kauai/lumahai.shp"       
[42] "E:/Data/AOI_Shapefiles/kauai/lumahai.shp"       
[43] "E:/Data/AOI_Shapefiles/maui/paukukalo.shp"      
[44] "E:/Data/AOI_Shapefiles/maui/paukukalo.shp"      
[45] "E:/Data/AOI_Shapefiles/maui/paukukalo.shp"      
[46] "E:/Data/AOI_Shapefiles/maui/paukukalo.shp"      
[47] "E:/Data/AOI_Shapefiles/oahu/pearl_harbor.shp"   
[48] "E:/Data/AOI_Shapefiles/oahu/pearl_harbor.shp"   
[49] "E:/Data/AOI_Shapefiles/oahu/pearl_harbor.shp"   
[50] "E:/Data/AOI_Shapefiles/oahu/pearl_harbor.shp"   
[51] "E:/Data/AOI_Shapefiles/oahu/pearl_harbor.shp"   
[52] "E:/Data/AOI_Shapefiles/oahu/pearl_harbor.shp"   
[53] "E:/Data/AOI_Shapefiles/maui/waihee.shp"         
[54] "E:/Data/AOI_Shapefiles/maui/waihee.shp"         
[55] "E:/Data/AOI_Shapefiles/maui/waihee.shp"         
[56] "E:/Data/AOI_Shapefiles/kauai/wailua_opaekaa.shp"
[57] "E:/Data/AOI_Shapefiles/kauai/wailua_opaekaa.shp"
[58] "E:/Data/AOI_Shapefiles/kauai/wailua_opaekaa.shp"
[59] "E:/Data/AOI_Shapefiles/kauai/wailua_opaekaa.shp"

In [ ]:
# wetlands_gpkg %>% plot

In [13]:
dir.create('HI_DEM')

In [14]:
#folder link to id
# https://drive.google.com/drive/folders/15nbWHXZdDOZbt9gUuB9BUPeCXc2SoivB?usp=drive_link
jp_folder = "15nbWHXZdDOZbt9gUuB9BUPeCXc2SoivB"
folder_id = drive_get(as_id(jp_folder))
print(folder_id)
#find files in folder
files = drive_ls(folder_id)

#loop dirs and download files inside them
for (i in seq_along(files$name)) {

  i_dir <- files[i, ]
    try({
      drive_download(
        as_id(i_dir$id[1]),
        path = paste0('/content/HI_DEM/', i_dir$name[1])
      )
    })
  }


# A dribble: 1 × 3
  name  id                                drive_resource   
  <chr> <drv_id>                          <list>           
1 dem   15nbWHXZdDOZbt9gUuB9BUPeCXc2SoivB <named list [34]>


File downloaded:

• maui_10m.tif <id: 10dHshukbiMC3z7h4xWxeOdY7x6dtkOf2>

Saved locally as:

• /content/HI_DEM/maui_10m.tif

File downloaded:

• kauai_10m.tif <id: 17G_F8uiM6c-0qe24JPCGUZWMAp2uoIro>

Saved locally as:

• /content/HI_DEM/kauai_10m.tif

File downloaded:

• oahu_10m.tif <id: 19_F1B-d5trv8iwBDBNKcLCWa19vnFvGI>

Saved locally as:

• /content/HI_DEM/oahu_10m.tif

File downloaded:

• molokai_10m.tif <id: 14WqA4pNafauzjr2tlJUUJjYROsaH7uEX>

Saved locally as:

• /content/HI_DEM/molokai_10m.tif



In [15]:
# spatial polygon of each dem making the footprints of each dem.
dem_gpkg <-
"/content/HI_DEM" %>%
  list.files(pattern='*.tif', full.names = T) %>%
  # lapply(stack) %>%
  # lapply(extent) %>%
  lapply(function(x){
    x_tif <- x %>% stack
    x_gpkg <- x_tif %>% extent %>% as('SpatialPolygons') %>% st_as_sf(crs = st_crs(x_tif))
    x_gpkg$path <- x
    st_crs(x_gpkg) <- st_crs(x_tif)
    x_gpkg %>% data.table %>% return
    }) %>%
  rbindlist()

In [16]:
dir.create('HI_CCAP')

In [17]:
#folder link to id
# https://drive.google.com/drive/folders/14fLGpMg2n9o6c8YmH4NSHc_ZQOC5MAuv?usp=drive_link
jp_folder = "14fLGpMg2n9o6c8YmH4NSHc_ZQOC5MAuv"
folder_id = drive_get(as_id(jp_folder))
print(folder_id)
#find files in folder
files = drive_ls(folder_id)

#loop dirs and download files inside them
for (i in seq_along(files$name)) {

  i_dir <- files[i, ]
    try({
      drive_download(
        as_id(i_dir$id[1]),
        path = paste0('/content/HI_CCAP/', i_dir$name[1])
      )
    })
  }


# A dribble: 1 × 3
  name  id                                drive_resource   
  <chr> <drv_id>                          <list>           
1 ccap  14fLGpMg2n9o6c8YmH4NSHc_ZQOC5MAuv <named list [34]>


File downloaded:

• kauai_ccap_2010.tif <id: 1-9SuvnH-Ck4CMaAXqDQTbbLGKtfMUnxn>

Saved locally as:

• /content/HI_CCAP/kauai_ccap_2010.tif

File downloaded:

• oahu_ccap_2010.tif <id: 1-5Q_qbss1zUnW1wcQlomRHehPemzy7HG>

Saved locally as:

• /content/HI_CCAP/oahu_ccap_2010.tif

File downloaded:

• molokai_ccap_2010.tif <id: 1-2M0JNgYiVMINDoW2mAiWLX3H9kYz_Z->

Saved locally as:

• /content/HI_CCAP/molokai_ccap_2010.tif

File downloaded:

• maui_ccap_2010.tif <id: 1--1nN7Gj_qx6iwPG8Jq5gZcGRA0uaOay>

Saved locally as:

• /content/HI_CCAP/maui_ccap_2010.tif



In [18]:
ccap_gpkg <-
"/content/HI_CCAP" %>%
  list.files(pattern='*.tif', full.names = T) %>%
  # lapply(stack) %>%
  # lapply(extent) %>%
  lapply(function(x){
    x_tif <- x %>% stack
    x_gpkg <- x_tif %>% extent %>% as('SpatialPolygons') %>% st_as_sf(crs = st_crs(x_tif))
    x_gpkg$path <- x
    st_crs(x_gpkg) <- st_crs(x_tif)
    x_gpkg %>% data.table %>% return
    }) %>%
  rbindlist()

In [ ]:
# dem_gpkg %>% st_as_sf %>% st_crs

In [19]:
dir.create('HI_2021_NAIP')

In [33]:
(runif(1)*10) %>% round

[1] 1

In [95]:
file.exists('/content/HI_2021_NAIP/m_2015604_se_04_060_20220108.tif')

[1] TRUE

In [66]:
download_index_gpkg[6,] %>% data.table()

ERROR: Error in parse(text = x, srcfile = src): <text>:2:26: unexpected symbol
1: # download_index_gpkg[6,] %>% data.table()
2: download_index_gpkg[6, 0].to_dataframe
                            ^


In [59]:
paste0("https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/",download_index_gpkg[5,]$location)

  download.file(
    paste0("https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/",download_index_gpkg[5,]$location),
    # x_dt$url,
     paste0('/content/HI_2021_NAIP/',download_index_gpkg[5,]$location,sep='') )

ERROR: Error in download_index_gpkg[5, ]: incorrect number of dimensions


In [68]:
download.file('https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/m_2115832_sw_04_060_20220211.tif')

ERROR: Error in download.file("https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/m_2115832_sw_04_060_20220211.tif"): argument "destfile" is missing, with no default


In [84]:
# URL of the file to download
file_url <- 'https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/m_2115733_sw_04_060_20221215.tif'

# Destination file path
dest_file <- 'm_2115733_sw_04_060_20221215.tif'

# Download the file with optimizations
download.file(url = file_url,
              destfile = dest_file,
              method = "libcurl")
              # extra = "-#",
              # options = list(timeout = 3600)) # Set a long timeout, e.g., 1 hour

# Note: The `extra = "-#"` option adds a progress bar for command line usage. If you're running this in an R environment
# that does not support this kind of progress bar (e.g., some RStudio versions), you might want to remove it or adjust accordingly.


In [85]:
# URL of the file to download
file_url <- 'https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/m_2115741_ne_04_060_20221215.tif'

# Destination file path
dest_file <- 'm_2115741_ne_04_060_20221215.tif'

# Download the file with optimizations
download.file(url = file_url,
              destfile = dest_file,
              method = "libcurl")

In [86]:
# URL of the file to download
file_url <- 'https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/m_2115741_nw_04_060_20221215.tif'

# Destination file path
dest_file <- 'm_2115741_nw_04_060_20221215.tif'

# Download the file with optimizations
download.file(url = file_url,
              destfile = dest_file,
              method = "libcurl")

In [87]:
# URL of the file to download
file_url <- 'https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/m_2115840_se_04_060_20221215.tif'

# Destination file path
dest_file <- 'm_2115840_se_04_060_20221215.tif'

# Download the file with optimizations
download.file(url = file_url,
              destfile = dest_file,
              method = "libcurl")

In [88]:
# URL of the file to download
file_url <- 'https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/m_2015604_se_04_060_20220108.tif'

# Destination file path
dest_file <- 'm_2015604_se_04_060_20220108.tif'

# Download the file with optimizations
download.file(url = file_url,
              destfile = dest_file,
              method = "libcurl")

In [89]:
# URL of the file to download
file_url <- 'https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/m_2015604_se_04_060_20221214.tif'

# Destination file path
dest_file <- 'm_2015604_se_04_060_20221214.tif'

# Download the file with optimizations
download.file(url = file_url,
              destfile = dest_file,
              method = "libcurl")

In [90]:
# URL of the file to download
file_url <- 'https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/m_2215962_ne_04_060_20220321.tif'

# Destination file path
dest_file <- 'm_2215962_ne_04_060_20220321.tif'

# Download the file with optimizations
download.file(url = file_url,
              destfile = dest_file,
              method = "libcurl")

In [91]:
# URL of the file to download
file_url <- 'https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/m_2215962_nw_04_060_20220321.tif'

# Destination file path
dest_file <- 'm_2215962_nw_04_060_20220321.tif'

# Download the file with optimizations
download.file(url = file_url,
              destfile = dest_file,
              method = "libcurl")

In [92]:
# URL of the file to download
file_url <- 'https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/m_2215962_se_04_060_20220321.tif'

# Destination file path
dest_file <- 'm_2215962_se_04_060_20220321.tif'

# Download the file with optimizations
download.file(url = file_url,
              destfile = dest_file,
              method = "libcurl")

In [93]:
# URL of the file to download
file_url <- 'https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/m_2215962_sw_04_060_20220321.tif'

# Destination file path
dest_file <- 'm_2215962_sw_04_060_20220321.tif'

# Download the file with optimizations
download.file(url = file_url,
              destfile = dest_file,
              method = "libcurl")

In [94]:
stoppdidue

ERROR: Error in eval(expr, envir, enclos): object 'stoppdidue' not found


In [96]:
cl <- makeCluster(8)
1:nrow(download_index_gpkg) %>%
  # parLapply(cl,.,function(x,download_index_gpkg){
  lapply(function(x){
  library(magrittr)
  x_dt <- download_index_gpkg[x,]
  Sys.sleep((runif(1)*10) %>% round)

  if(!file.exists(paste0('/content/HI_2021_NAIP/',x_dt$location,sep=''))){
   tryCatch({
  download.file(
    paste0("https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/",x_dt$location),
    # x_dt$url,
     paste0('/content/HI_2021_NAIP/',x_dt$location,sep='') )
   },error={return( paste0('/content/HI_2021_NAIP/',x_dt$location,sep=''))})
  }
  # },download_index_gpkg)
  })
stopCluster(cl)

ERROR: Error in 1:nrow(download_index_gpkg): argument of length 0


In [ ]:
# Load necessary libraries
library(parallel)
library(magrittr)
# Assuming download_index_gpkg is a vector of file names or URLs
download_index_gpkg <- c("m_2115832_sw_04_060_20220211.tif", "m_2215953_sw_04_060_20220321.tif") # Replace with your actual files or URLs

# Define the base URL if the files are identified by names that need to be appended to a base URL
base_url <- "https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/"

# Define the directory where files will be saved
save_directory <- "/content/HI_2021_NAIP"

# Ensure the save directory exists
if(!dir.exists(save_directory)) {
  dir.create(save_directory, recursive = TRUE)
}
library(magrittr)
# Define the download function with checks for existing files and randomized sleep
download_file <- function(file_name) {
  # Full URL of the file to download
  full_url <- paste0(base_url, file_name)

  # Destination file path
  dest_file_path <- paste0(save_directory, file_name)

  # Check if the file already exists
  if(!file.exists(dest_file_path)) {
    # Randomized sleep to reduce load on the server
    Sys.sleep(runif(1, 0, 10) %>% round)

    # Download the file
    download.file(full_url, dest_file_path, mode = "wb")

    return(paste("Downloaded", file_name))
  } else {
    return(paste("Skipped", file_name, "- already exists"))
  }
}

# Set up parallel computing
num_cores <- 8
cl <- makeCluster(num_cores)
library(magrittr)
# Export necessary variables and functions to the cluster
clusterExport(cl, varlist = c("download_file", "base_url", "save_directory"), envir = environment())

# Execute the download in parallel
result <- parLapply(cl, download_index_gpkg, download_file)

# Stop the cluster
stopCluster(cl)

# Print result to see which files were downloaded or skipped
print(result)


In [ ]:
download.file('https://coast.noaa.gov/htdata/raster7/imagery/HI_NAIP_2021_9668/HI_NAIP_2021_EPSG-26904.vrt', paste('/content/HI_2021_NAIP/HI_NAIP_2021_EPSG-26904.vrt',sep=''))

In [ ]:
hawaii_naip <- stack('/content/HI_2021_NAIP/HI_NAIP_2021_EPSG-26904.vrt')

In [ ]:
dir.create('output_fishnets')

In [ ]:
# go through each wetland, one at a time. wetlands_sample is imagery dataset.
# translate dem gpkg

1:nrow(wetlands_gpkg) %>%
  lapply(function(y){
wetlands_sample <- wetlands_gpkg[y,] %>% st_transform(st_crs(hawaii_naip)) %>% crop(hawaii_naip,.)
dem_loc_gpkg <- dem_gpkg %>% st_as_sf %>% st_transform(st_crs(wetlands_gpkg[y,])) %>% st_intersection(wetlands_gpkg[y,])
dem_load_tif <- dem_loc_gpkg$path %>% raster %>% crop(., wetlands_fishnnet_gpkg[y,] %>% st_transform(st_crs(dem_load_tif)))
nlcd_load_tif <- nlcd_tif %>% crop(., wetlands_fishnnet_gpkg[y,] %>% st_transform(st_crs(nlcd_tif)))
wetlands_fishnnet_gpkg <- wetlands_gpkg[y,] %>% st_transform(st_crs(hawaii_naip)) %>% st_make_grid(cellsize =c(76.8,76.8)) %>% st_as_sf
# file.remove(paste0('/content/output_fishnets/',wetlands_sample$layer[1],'.gpkg'))
st_write(wetlands_fishnnet_gpkg,paste0('/content/output_fishnets/',wetlands_gpkg$layer[y],'.gpkg'),delete_dsn=T,delete_layer = T)

cl <- makeCluster(8)
1:nrow(download_index_gpkg) %>%
  parLapply(cl,.,function(x,wetlands_fishnnet_gpkg,hawaii_naip,wetlands_sample,dem_load_tif,nlcd_load_tif,nwi_gpkg){

library(raster)
library(sf)
library(fasterize)
library(magrittr)
library(googledrive)
library(parallel)
library(data.table)
sample_chip <- wetlands_fishnnet_gpkg[x,] %>% st_transform(st_crs(hawaii_naip)) %>% crop(wetlands_sample,.)
sample_chip$dem <- projectRaster(dem_load_tif, sample_chip, method="ngb")
sample_chip$nlcd <- projectRaster(nlcd_load_tif, sample_chip, method="ngb")
nwi_gpkg$wetland_ty <- nwi_gpkg$wetland_ty %>% as.factor
nwi_gpkg <- nwi_gpkg %>% st_transform(st_crs(sample_chip))
sample_chip$class <- fasterize(
nwi_gpkg,
sample_chip[[1]],
field = 'wetland_ty',
fun = "first")
sample_chip$class_fill <-sample_chip$class
values(sample_chip$class_fill)[is.na(values(sample_chip$class_fill))] <- values(sample_chip$nlcd)[is.na(values(sample_chip$class_fill))]
values(sample_chip$class_fill)[is.na(values(sample_chip$class_fill))] <- 0
values(sample_chip$dem)[is.na(values(sample_chip$dem))] <- 0
values(sample_chip$nlcd)[is.na(values(sample_chip$nlcd))] <- 0
values(sample_chip$class)[is.na(values(sample_chip$class))] <- 0
sample_chip  %>% writeRaster(paste0('output_chips/',(sample_chip %>% extent %>% round)[1],'_',(sample_chip %>% extent %>% round)[3],'.tif'),overwrite=T)

  },wetlands_fishnnet_gpkg,hawaii_naip,wetlands_sample,dem_load_tif,nlcd_load_tif,nwi_gpkg)
stopCluster(cl)
  })

Zip files and save to drive

In [ ]:
# zip(zipfile = 'testZip', files = 'testDir/test.csv') #make this work
zip(zipfile = "/content/testZip.zip",
    files = c("/content/testZip"))

In [ ]:
#put code here to upload to drive
# save to processed_data https://drive.google.com/drive/folders/1CXLo3IfmEVgOIaKHtpaXiblhBePgl1fS?usp=drive_link
drive_upload(media = "/content/testZip.zip", path = as_id("1CXLo3IfmEVgOIaKHtpaXiblhBePgl1fS"))